A number may be divided by two an arbitrarily large number of times using the standard collatz function. Rather than factoring
these many halving operations into a given number's depth or orbit length, we can redefine the collatz function to separate
these excess operations into a new dimension.

Define the smart collatz path as such (based off the odd collatz function):

    if x mod 8 = 5, (x-1)/4, "L"
    if x mod 4 = 3, (3x+1)/2, "2"
    if x mod 8 = 1, (3x+1)/4, "4"
    
where we can keep track of which cases have been used by appending the corresponding characters to a running string.

For example, f(11) = "24L2L", since:
    
    11 mod 4 = 3, (3(11)+1)/2 = 17, append "2"
    17 mod 8 = 1, (3(17)+1)/4 = 13, append "4"
    13 mod 8 = 5, (13-1)/4 = 3, append "L"
    3 mod 4 = 3, (3(3)+1)/4 = 5, append "2"
    5 mod 8 = 5, (5-1)/4 = 1, append "L"
    
Define a number's depth as the number of times "2" or "4" appears in its path.

Objective: Find the finite state machine at any given depth that generates all numbers of the odd collatz set at that depth. Nest the state machines iteratively until depth = 1.

Interesting observations:
                                                                                  
    -number of starting nodes in each depth cycle is 2 * 3 ** (depth - 2)
    -every cycle ends with 4*, the same as the 1 -> 1 cycle
    -every node warps to one of next depth without exception, and nodes at lesser depth cannot warp to higher nodes
    -distribution of warps starts with 4 pointing to 0, 2 pointing to 1, 4 pointing to 2...
    
Further inquiry:
                                                                                  
    -possible to section based off machine?
    -discernable patterns between state transitions at different depths?
    -helpful for proving the conjecture?

In [1]:
#binary shortcuts

def binary(n):
    "Bin function, but removes the leading 0b."
    return str(bin(n))[2:]

def binaryAdd(str1, str2):
    "Adds two binary strings and returns the sum string."
    if not str1:
        return str2
    if not str2:
        return str1
    return binary(int(str1, 2) + int(str2, 2))

def binaryMultiply(str1, str2):
    "Multiplies two binary strings and returns the product string."
    if str1 and str2:
        return binary(int(str1, 2) * int(str2, 2))
    return ""

In [2]:
#implement the smart collatz traversal

def collDirection(string):
    "Performs one pass of the smart collatz function and returns a list containing the modified binary and direction used."
    
    direction = ""
    if len(string) > 2 and string[-3:] == "101":
        direction = "L"
        string = string[:-3] + "1"
    elif len(string) > 1 and string[-2:] == "11":
        direction = "2"
        string = binaryAdd(binaryMultiply(string[:-2], "11"), "10") + "1"
    elif len(string) > 2 and string[-3:] == "001":
        direction = "4"
        string = binaryMultiply(string[:-3], "11") + "1"
    return [string, direction]

def easyColl(string, verbose=False):
    "Returns the smart collatz orbit of the number as a string with appended directions."
    
    history = ""
    if verbose:
        print(int(string, 2))
        
    #repeatedly call collDirection on results of previous call
    while string is not "1":
        data = collDirection(string)
        if verbose:
            print(int(data[0], 2), data[1])
        string = data[0]
        history += data[1]
    return history

In [3]:
#identify depth identities

def smallBranches(n, depth, base=1):
    "Returns a list containing the n closest numbers at specific depth from 1. Skips branches with multiples of 3 as roots implicitly."
    
    if depth == 0:
        return []
    
    #generate n numbers of the depth 1 set
    x = [base]
    for i in range(1, n):
        if (4 * x[i-1] + 1) % 3 != 0:
            x.append(4 * x[i-1] + 1)
        else:
            x.append(16 * x[i-1] + 5)
            
    #iteratively update each member to depth + 1 by testing multiples of 2, moving right (sidetracking) at multiples of 3.
    for i in range(1, depth):
        for j, branch in enumerate(x):
            if branch % 3 == 0:
                x[j] = 4 * branch + 1
                branch = x[j]
            exp = 1
            while (branch * (2 ** exp) - 1) % 3 != 0:
                exp += 1
            x[j] = (branch * (2 ** exp) - 1)//3
            
    return x

In [4]:
#derive finite automata transitions

def highlightDifference(str1, str2):
    "Prints a string labelling the binary and path differences between consecutive depth identities."
    
    idx = 0
    while idx < len(str1) and idx < len(str2) and str1[idx] is str2[idx]:
        idx += 1
    firstStrip = easyColl(str1 + "1").strip('L')
    pad = 'L'
    if firstStrip[-1] == '2':
        pad = 'LL'
    print(firstStrip + " (" + str1[idx:] + ") -> " + easyColl(str2 + "1").strip('L') + pad + " (" + str2[idx:] + ")")

def printDifferences(depth):
    "Prints all unique differences at specified depth."
    
    print("Transition Difference Info:\n")
    
    cycle = smallBranches(2 * 3 ** (depth-2) + 2, depth)
    print(0, end = " ")
    highlightDifference(binary(cycle[-2])[:-1], binary(cycle[-1])[:-1])
    i = 1
    while i + 1 < len(cycle) - 1:
        print(i, end = " ")
        highlightDifference(binary(cycle[i])[:-1], binary(cycle[i + 1])[:-1])
        i += 1

In [5]:
#match warp nodes at state transitions to "nest" machines

def matchWarps(depth):
    "Returns list containing which nodes at depth-1 each depth node warps to."
    
    warps = []
    cycle1 = smallBranches(2 * 3 ** (depth-2) + 1, depth)
    cycle2 = smallBranches(2 * 3 ** (depth-3) + 1, depth-1)
    
    #map identity to node index for depth - 1
    tdict = {}
    for idx in range(1, len(cycle2)):
        ecstr = easyColl(binary(cycle2[idx]))
        count = 0
        j = -1
        while count < depth - 2:
            j += 1
            if ecstr[j] == '2' or ecstr[j] == '4':
                count += 1
        tdict[ecstr[:j+1]] = idx % (len(cycle2) - 1)
    
    #extract parallel identity at depth and print matching depth - 1 indices 
    for i in ([len(cycle1) - 1] + list(range(1, len(cycle1) - 1))):
        ecstr = easyColl(binary(cycle1[i]))
        count = 0
        j = -1
        while count < depth - 2:
            j += 1
            if ecstr[j] == '2' or ecstr[j] == '4':
                count += 1
        #print(i % (len(cycle1) - 1), tdict[ecstr[:j+1]])
        warps.append(tdict[ecstr[:j+1]])
        
    return warps

With condensed state differences and warp information, we can now construct the state machine at depth n.

To read the output:
    
    Assign warp node to transition to as ("depth - 1" + matchWarps output).
    Label depth node to warp node transition according to number in 1st parentheses.
    Label depth node to depth node transition as (number in 2nd parentheses - number in first parentheses for next node).
        If impossible to subtract, carry over from the previous node's 2nd parentheses

Example at depth 4:
    
    node 4_0 transitions with "0" to node 3_0 and with "1" to node 4_1
    node 4_1 transitions with "01" to node 3_2 and with "10010" to node 4_2
    node 4_2 transitions with "00" to node 3_1 and with "" to node 4_3
    node 4_3 transitions with "0" to node 3_5 and with "1001000" to node 4_4
    node 4_4 transitions with "00" to node 3_0 and with "" to node 4_5
    node 4_5 transitions with "0" to node 3_0 and with "101" to node 4_6
    node 4_6 transitions with "01" to node 3_0 and with "10000" to node 4_7
    node 4_7 transitions with "0" to node 3_0 and with "11" to node 4_8
    node 4_8 transitions with "" to node 3_0 and with "1" to node 4_9
    node 4_9 transitions with "" to node 3_0 and with "111001" to node 4_10
    node 4_10 transitions with "01" to node 3_0 and with "10" to node 4_11
    node 4_11 transitions with "00" to node 3_0 and with "1011" to node 4_12
    node 4_12 transitions with "00" to node 3_0 and with "11" to node 4_13
    node 4_13 transitions with "000" to node 3_0 and with "10" to node 4_14
    node 4_14 transitions with "0" to node 3_0 and with "100" to node 4_15
    node 4_15 transitions with "0" to node 3_0 and with "111" to node 4_16
    node 4_16 transitions with "011000" to node 3_0 and with "" to node 4_17
    node 4_17 transitions with "01" to node 3_0 and with "" to node 4_18

In [6]:
#print all information needed to define the state machine at depth n

depth = 4
printDifferences(depth)
print()
print("Warp Node Info:\n")
print(matchWarps(depth))

Transition Difference Info:

0 444 (0) -> 24L2L (101)
1 24L2 (01) -> 4L24LL (1001000)
2 4L24 (0) -> 422L ()
3 422 (0) -> 44L4LL (100100000)
4 44L4 (0) -> 442L ()
5 442 (0) -> 244LL (10101)
6 244 (01) -> 44L2L (100000)
7 44L2 (0) -> 224LL (11)
8 224 () -> 222L (1)
9 222 () -> 24L4LL (11100101)
10 24L4 (01) -> 4L42L (1000)
11 4L42 (00) -> 4L44LL (1011000)
12 4L44 (00) -> 4L4L2L (11000)
13 4L4L2 (000) -> 424LL (100)
14 424 (0) -> 4L22L (1000)
15 4L22 (0) -> 4L4L4LL (111011000)
16 4L4L4 (1000) -> 242L ()
17 242 (01) -> 444LL (10000000)

Warp Node Info:

[0, 2, 1, 5, 0, 0, 2, 0, 3, 3, 2, 4, 4, 4, 5, 1, 4, 2]
